In [89]:
import numpy as np
import pandas as pd
import os

from plotly import __version__
import cufflinks as cf
from plotly.offline import download_plotlyjs,init_notebook_mode,plot,iplot
init_notebook_mode(connected=True)
cf.go_offline()

%matplotlib inline

#user specify instrument and contract month

product=''
month=''

while True:
    inst=int(raw_input('Please input the number of the product you want to see the seasonals: 1) Brent 2) Gasoil 3) HO 4) RBOB 5) WTI: '))
    
    if inst==1:
        product='Brent'
        break
    elif inst==2:
        product='Gasoil'
        break   
    elif inst==3:
        product='HO'
        break      
    elif inst==4:
        product='RBOB'
        break       
    elif inst==5:
        product='WTI'
        break       
    else:
        print 'You did not input a correct number. Please try again.'

while True:
    Mth=int(raw_input('Please input the month of the contract which you wish to evaluate in numbers from 1-12: '))
    
    if Mth==1:
        month='Jan'
        break
    elif Mth==2:
        month='Feb'
        break   
    elif Mth==3:
        month='Mar'
        break   
    elif Mth==4:
        month='Apr'
        break   
    elif Mth==5:
        month='May'
        break          
    elif Mth==6:
        month='Jun'
        break   
    elif Mth==7:
        month='Jul'
        break   
    elif Mth==8:
        month='Aug'
        break   
    elif Mth==9:
        month='Sep'
        break   
    elif Mth==10:
        month='Oct'
        break   
    elif Mth==11:
        month='Nov'
        break   
    elif Mth==12:
        month='Dec'
        break 
    else:
        print 'You did not input a correct number. Please try again.'
        

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


Please input the number of the product you want to see the seasonals: 1) Brent 2) Gasoil 3) HO 4) RBOB 5) WTI: 5
Please input the month of the contract which you wish to evaluate in numbers from 1-12: 7


In [90]:
cd Quandl

C:\Users\boonsin.teo\Documents\Quandl


In [91]:
cd $product

C:\Users\boonsin.teo\Documents\Quandl\WTI


In [92]:
cd $month

C:\Users\boonsin.teo\Documents\Quandl\WTI\Jul


In [93]:

count=0
path = os.getcwd()

#indicate evaluation dates

start_mth=raw_input('Please input start mth of seasonal period in MM format: ')
start_day=raw_input('Please input start day of seasonal period in DD format: ')

i=int(raw_input('Please input 1 if seasonal period start on same year as contract year and 0 if not: '))
            
end_mth=raw_input('Please input end mth of seasonal period in MM format: ')
end_day=raw_input('Please input end day of seasonal period in DD format: ')

j=int(raw_input('Please input 1 if seasonal period ends on same year as contract year and 0 if not: '))

pos=1

while True:

    a=raw_input("Please input 'y' if position is long and 'n' if position is short: ").lower()
    
    if a=='y':
        break
    if a=='n':
        pos=pos*(-1)
        break
    else:
        print "Please input either 'y' or 'n' for position"

print 'PnL will be calculated basis 1kb lot size for standardization purposes.'

listing = os.listdir(path)
for infile in listing:
    temp=pd.read_excel(infile) #read each file into panda dataframe
    
    contract_yr_str=infile[-8:-4] # get the contract year
    
    seasonal_year=int(contract_yr_str)-1 # get the seasonal year to create the index - prev year
    seasonal_year=str(seasonal_year) # convert to string
    
    start_date=str(int(seasonal_year)+i)+'-'+start_mth+'-'+start_day # create start date
    end_date=str(int(seasonal_year)+j)+'-'+end_mth+'-'+end_day # create end date
    
    dates=pd.date_range(start_date, end_date) # range of dates for date axis
    dates=dates.strftime('%Y-%m-%d') #make sure no left out dates
    sample_data=np.arange(0,len(dates)) #just sample numbers to create dataframe
    
    dates_dataframe=pd.DataFrame({'Date':dates,'data':sample_data}) # create dates dataframe
    dates_dataframe=dates_dataframe.set_index('Date') # set index as Date to facilitate joining
    
    temp=temp.set_index('Date') # set index as Date to facilitate joining
    
    temp=dates_dataframe.join(temp) # join dataframes together on Date
   

    temp.reset_index(inplace=True)
    
    temp=temp[temp['Date']>=start_date] # filter for start date

    temp=temp[temp['Date']<=end_date] # filter for end date
 

    temp=temp.set_index('Date') # creating new index as Date
    
    
    temp=temp.reset_index() # putting date back out into col
    temp['Date']=pd.to_datetime(temp['Date']).dt.strftime('%m/%d')
    temp=temp.set_index('Date')
    temp=pd.DataFrame(temp['Settle'],columns=['Settle']) 

    #combine into seasonal index table
    
    if count==0:
        Seasonal_Combi=temp
        seasonal_year=str(int(seasonal_year)+1)
        Seasonal_Combi.columns=[seasonal_year]
        
        count+=1
        
    else:
        
        seasonal_year=str(int(seasonal_year)+1)
        Seasonal_Combi[seasonal_year]=temp['Settle']

#remove columns that are all NA
Seasonal_Combi.dropna(axis=1,how='all',inplace=True)

#remove rows that are all NA
Seasonal_Combi.dropna(axis=0,how='all',inplace=True)

#fillna values with values of forward value
Seasonal_Combi.fillna(method='bfill',inplace=True)
#fillna values with values of back value
Seasonal_Combi.fillna(method='ffill',inplace=True)

#Items to Create
#PnL - use last value vs first value

z=1
factor=1000
if product =='RBOB':
    z=42
elif product=='HO':
    z=42
elif product=='Gasoil':
    factor=100
else:
    z=1
    factor=1000

pnl_series=(Seasonal_Combi.iloc[-1]-Seasonal_Combi.iloc[0])*z*pos*factor    

# Win %
Win_Percent=float(pnl_series[pnl_series>0].count())/float(pnl_series.count())*100
    
#Largest Drawdown - use max and min
max_series=Seasonal_Combi.max()
min_series=Seasonal_Combi.min()
if pos==1:
    max_drawdown=(min_series-Seasonal_Combi.iloc[0])*pos*z*factor
else:
    max_drawdown=(max_series-Seasonal_Combi.iloc[0])*pos*z*factor
    
#% return - calculate using last value vs first value - similar to pnl calc
percent_return=((Seasonal_Combi.iloc[-1]-Seasonal_Combi.iloc[0])/(Seasonal_Combi.iloc[0]))


Please input start mth of seasonal period in MM format: 01
Please input start day of seasonal period in DD format: 15
Please input 1 if seasonal period start on same year as contract year and 0 if not: 1
Please input end mth of seasonal period in MM format: 04
Please input end day of seasonal period in DD format: 10
Please input 1 if seasonal period ends on same year as contract year and 0 if not: 1
Please input 'y' if position is long and 'n' if position is short: y
PnL will be calculated basis 1kb lot size for standardization purposes.


In [94]:
cd ..

C:\Users\boonsin.teo\Documents\Quandl\WTI


In [95]:
cd ..

C:\Users\boonsin.teo\Documents\Quandl


In [96]:
cd ..

C:\Users\boonsin.teo\Documents


In [97]:
Seasonal_Combi.head()

,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,...,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
Date,,,,,,,,,,,,,,,,,,,,,
01/15,17.32,23.17,17.35,12.75,24.67,26.25,20.05,28.50,30.73,47.12,...,90.41,53.80,80.86,95.43,101.77,94.68,92.44,49.85,33.86,55.73
01/16,16.99,22.68,17.53,12.69,24.67,26.25,20.24,28.80,31.98,47.12,...,89.09,52.79,81.53,95.43,101.77,95.54,92.13,52.09,33.31,55.73
01/17,17.35,22.43,17.39,12.69,24.67,25.80,19.52,28.96,32.60,47.12,...,88.30,49.64,81.53,95.43,101.77,96.77,92.70,49.68,33.31,55.73
01/18,17.42,22.47,17.39,12.69,24.67,26.28,19.56,28.90,32.60,47.12,...,88.53,49.64,81.53,95.43,101.52,97.00,93.03,49.68,33.31,54.35
01/19,17.29,22.47,17.39,12.69,24.63,27.50,19.84,28.90,32.60,46.63,...,87.77,49.64,81.53,95.23,101.52,97.68,93.03,49.68,33.31,54.45


In [98]:
pnl_series

1996     3270.0
1997    -3460.0
1998    -1140.0
1999     3780.0
2000    -1620.0
2001     2520.0
2002     6020.0
2003    -2160.0
2004     5320.0
2005     8240.0
2006     2660.0
2007    12510.0
2008    18520.0
2009     3100.0
2010     5660.0
2011    18430.0
2012      340.0
2013      540.0
2014     8950.0
2015     4940.0
2016     8000.0
2017    -1930.0
dtype: float64

In [99]:
percent_return

1996    0.188799
1997   -0.149331
1998   -0.065706
1999    0.296471
2000   -0.065667
2001    0.096000
2002    0.300249
2003   -0.075789
2004    0.173121
2005    0.174873
2006    0.038974
2007    0.230982
2008    0.204845
2009    0.057621
2010    0.069998
2011    0.193126
2012    0.003341
2013    0.005703
2014    0.096820
2015    0.099097
2016    0.236267
2017   -0.034631
dtype: float64

In [100]:
max_drawdown

1996     -480.0
1997    -3870.0
1998    -3000.0
1999     -830.0
2000    -1620.0
2001     -450.0
2002     -530.0
2003    -2590.0
2004     -410.0
2005     -950.0
2006    -6500.0
2007        0.0
2008    -4730.0
2009   -11360.0
2010    -7890.0
2011    -4030.0
2012    -3860.0
2013    -3380.0
2014     -310.0
2015    -1370.0
2016    -1320.0
2017    -7100.0
dtype: float64

In [101]:
print 'The win percentage is %.2f percent' %Win_Percent

The win percentage is 77.27 percent


In [102]:
#Plot Equity Curve - use plot below for all columns

Equity_Curve=Seasonal_Combi.diff(axis=0)
Equity_Curve.iloc[0]=0.00
i=1
while i<len(Equity_Curve):
    Equity_Curve.iloc[i]=Equity_Curve.iloc[i-1]+Equity_Curve.iloc[i]*z*pos*factor
    i+=1

Equity_Curve.iplot(kind='line')

In [103]:
pwd

u'C:\\Users\\boonsin.teo\\Documents'